# Topic 17b: Unsupervised Learning

This is the code for Topic 17 that wouldn't run on a mac.

## K-means algorithm

The first issue arose when we tried to run the K-means clustering algorithm.

Here is the algorithm for the K-means clustering:

1. Initialize the number _K_, which is the number of clusters to find.
1. Initialize the centroids (the location of the center) of the _K_ clusters.
1. Decide the class memberships of the _N_ samples: for each sample, place it into the cluster that has the closest centroid.
1. Revise the centroids of the clusters: each centroid moves to the mean of the locations for all the samples in that cluster.
1. If none of the samples changed clusters in the previous step, the algorithm terminates.  Otherwise, go bad to step 3.

As usual, we install the packages we will need in this notebook (although we can add more later):



In [ ]:
%matplotlib inline
import numpy as np
from sklearn import cluster
import matplotlib.pylab as plt

Now we will create some data.  We will generate three clusters of 40 points each.  We center the points for these clusters around these locations: [0,0], [5,5], and [8,3].  These will be in the X array, and each entry will have two values: the x- and y-coordinate.

We will also build an 'answer' array, y.  This array indicates the cluster membership for each sample, with the first 40 samples being in cluster 1, the next 40 in cluster 2, and the remaining in cluster 3.  Normally, we would not have this data (for unsupervised learning we don't have the 'answers').  However, to help us see what is going on, we create this data.  We won't pass these values to the machine learning code, but we will use this to color some of our plots.


In [ ]:
MAXN = 40
X = np.concatenate([1.25*np.random.randn(MAXN,2), 5 + 1.5*np.random.randn(MAXN,2)])
X = np.concatenate([X, [8,3] + 1.2*np.random.randn(MAXN, 2)])

y = np.concatenate([np.ones((MAXN,1)),2*np.ones((MAXN,1))])
y = np.concatenate([y,3*np.ones((MAXN,1))])

In the following two plots, we see the data.  In the left plot, we have colored the points to show which cluster they were created within.  The right plot shows the data that the algorithm sees: Just the points, with no clue as to the coloring.

In [ ]:
plt.rc('figure', figsize = (12, 5))
plt.subplot(1,2,1)
plt.scatter(X[(y==1).ravel(),0],X[(y==1).ravel(),1],color='r')
plt.scatter(X[(y==2).ravel(),0],X[(y==2).ravel(),1],color='b')
plt.scatter(X[(y==3).ravel(),0],X[(y==3).ravel(),1],color='g')
plt.title('Data as were generated')

plt.subplot(1,2,2)
plt.scatter(X[:,0],X[:,1],color='r')
plt.title('Data as the algorithm sees them')
plt.show()

In the following we will do the actual clustering.  There are techniques that can help us determine the number of clusters to use, but for now we will simply say that there should be 3 clusters.

In [ ]:
K = 3
clf = cluster.KMeans(init='random', n_clusters=K, n_init=10, max_iter=300, random_state=12)
clf.fit(X)

* "init='random'" tells the algorithm to randomize the initial locations of the centroids..
* "n_clusters" sets the number of clusters.
* "n_init" says to run the whole process 10 times.  The _K-means_ clusterizer can get stuck in local minima, so by running multiple times we try to avoid these less-optimal solutions.
* "max_iter": Recall that _K-means_ iteratively refines the centroids of the clusters.  The process stops when the samples do not change clusters.  However, sometimes the process may iterate with very minimal changes, so this factor can be used to make sure the process terminates.
* "random_state" initializes the random number generator, as was mentioned in other lectures.

The *labels_* attribute of the clustering method indicates the final answer, showing for each sample which cluster it was assigned:

In [ ]:
print(clf.labels_)

Ideally, the first 40 points would all belong to one cluster, the next 40 in a second cluster, and the remaining points in the third cluster.  We see, however, there are a few points that are assigned to the wrong cluster.

Let's look again at the plot:

In [ ]:
plt.scatter(X[(y==1).ravel(),0],X[(y==1).ravel(),1],color='r')
plt.scatter(X[(y==2).ravel(),0],X[(y==2).ravel(),1],color='b')
plt.scatter(X[(y==3).ravel(),0],X[(y==3).ravel(),1],color='g')
fig = plt.gcf()
fig.set_size_inches((6, 5))

We can see that the blue and green clusters overlap, so there are some green points deep within the blue territory, and a blue dot close to the center of the green.  Without the coloring information, how is the clustering program supposed to know the correct placement of these colors?  We cannot expect the program to get 100% accuracy!

What we want to do next is to draw the territories for each of the clusters.  How do we do that?  The technique they use is this:

* Build a _mesh grid_, an array of 200 x 200 points covering the area of our drawing.  
* Take all of those grid points and run them through the trained clustering algorithm.  This will then indicate the cluster that would be assigned to each of the grid points.  
* Use _implot_ to plot the mesh "image".  This plot will be drawn a little blurred, so the individual points merge.  
* Draw the original data points over the top.

In [ ]:
x = np.linspace(-5,15, 200)
XX, YY = np.meshgrid(x, x)
sz = XX.shape
data = np.c_[XX.ravel(), YY.ravel()]
Z = clf.predict(data)

In [ ]:
plt.imshow(Z.reshape(sz), interpolation='bilinear', origin='lower', extent=(-5, 15, -5, 15), alpha=0.3, vmin=0, vmax=K-1)
plt.title('Space partitions', size=14)
plt.scatter(X[(y==1).ravel(),0],X[(y==1).ravel(),1],color='r')
plt.scatter(X[(y==2).ravel(),0],X[(y==2).ravel(),1],color='b')
plt.scatter(X[(y==3).ravel(),0],X[(y==3).ravel(),1],color='g')
fig = plt.gcf()
fig.set_size_inches((6, 5))
plt.show()

The plot shows that the clusters were separated quite nicely, and we can also see that a few of the points are 'mis-categorized'.  Again, the program actually did the right thing, those 'incorrect' points are really inside the limits of the other cluster.

In [ ]:
clf = cluster.KMeans(n_clusters=K, random_state=0)
clf.fit(X)

data=np.c_[XX.ravel(),YY.ravel()]
Z=clf.predict(data)

In [ ]:
plt.title('Final result of K-means', size=14)

plt.scatter(X[(y==1).ravel(),0],X[(y==1).ravel(),1],color='r')
plt.scatter(X[(y==2).ravel(),0],X[(y==2).ravel(),1],color='b')
plt.scatter(X[(y==3).ravel(),0],X[(y==3).ravel(),1],color='g')

plt.imshow(Z.reshape(sz), interpolation='bilinear', origin='lower', extent=(-5, 15, -5, 15), alpha=0.3, vmin=0, vmax=K-1)

x = np.linspace(-5,15, 200)
XX, YY = np.meshgrid(x, x)

fig = plt.gcf()
fig.set_size_inches((6, 5))

plt.show()

The _K-means_ algorithm clusters the data by finding the optimal _centroids_ for the clusters.  It chooses these centroids by minimizing a criterion known as the _inertia_, which is the sum-of-squares distance from each point of a cluster to its centroid.

This inertia is thought of as a measure of the compactness of the cluster, or as 'a measure of how internally coherent cluster are'.

We can view some metrics of the clustering:

In [ ]:
from sklearn import metrics

In [ ]:
def test_cluster(n_clusters, init, max_iter, n_init):
  clf = cluster.KMeans(n_clusters=n_clusters, init=init, random_state=0, max_iter=max_iter, n_init=n_init)
  clf.fit(X)
  print('Final evaluation of the clustering:')
  print(f'Inertia: {clf.inertia_:.2f}')
  print(f'Adjusted_rand_score: {metrics.adjusted_rand_score(y.ravel(), clf.labels_):.2f}')
  print(f'Homogeneity: {metrics.homogeneity_score(y.ravel(), clf.labels_):.2f}')
  print(f'Completeness: {metrics.completeness_score(y.ravel(), clf.labels_):.2f}')
  print(f'V_measure: {metrics.v_measure_score(y.ravel(), clf.labels_):.2f}')
  print(f'Silhouette: {metrics.silhouette_score(X, clf.labels_, metric="euclidean"):.2f}')
  plt.title(f'K-means, {n_clusters} clusters, {max_iter} iterations, {n_init} n_init, {init}', size=14)
  plt.scatter(X[(y==1).ravel(),0],X[(y==1).ravel(),1],color='r')
  plt.scatter(X[(y==2).ravel(),0],X[(y==2).ravel(),1],color='b')
  plt.scatter(X[(y==3).ravel(),0],X[(y==3).ravel(),1],color='g')
  x = np.linspace(-5,15, 200)
  XX, YY = np.meshgrid(x, x)
  data=np.c_[XX.ravel(),YY.ravel()]
  Z=clf.predict(data)
  plt.imshow(Z.reshape(sz), cmap='hsv', interpolation='bilinear', origin='lower', extent=(-5, 15, -5, 15), alpha=0.6, vmin=0, vmax=n_clusters-1)
  fig = plt.gcf()
  fig.set_size_inches((6, 5))
  plt.show()

test_cluster(3, 'k-means++', 300, 10)

In [ ]:
test_cluster(3, 'random', 2, 2)

In [ ]:
test_cluster(3, 'k-means++', 1, 1)

In [ ]:
test_cluster(8, 'k-means++', 300, 10)

Here are some issues with k-means:

* Inertia makes the assumption that the clusters are convex and isotropic, which is not always the case.  It responds poorly to elongated clusters, or manifolds with irregular shapes.

* Given enough time, K-means will always converge, although it may be converging on a local minima.

* The algorithm requires the number of clusters to be specified.

* It scales well to a large number of samples and has been used across a large range of application areas in many different fields.

* The computation ios often done several times (with an n_init > 1), with different initializations of the centroids.  This is to help alleviate the local minima issue.

* The _k-means++_ initialization scheme initializes the centroids to be generally distant from each other, leading to better results than the _random_ initialization scheme.


## Spectral Clustering

The k-means clustering groups the data to optimize "compactness".

Spectral Clustering groups the data to optimize "connectivity".

For example, Spectral Clustering might compute a _similarity matrix_ giving a measure of the similarity between any two samples in the dataset.  The algorithm then merges or clusters the samples that are the closest together.

The expectation is that two points in the same cluster are close together.  However, two distant points might be in the same cluster if there is a 'path' between them (of other samples), such that the distance between subsequent pairs of samples on the path are small.

Scikit-learn has some tools in its library to generate datasets for use in illustrating these algorithms:

In [ ]:
from sklearn.datasets import make_blobs, make_moons

X, labels_true = make_blobs(n_samples = 1000, centers = 3, cluster_std=[1.7,1.7,1.7])
[Xmoons, ymoons] = make_moons(n_samples=300, noise=0.05)

plt.subplot(1,2,1)
plt.scatter(X[:, 0], X[:, 1], c='r', marker='o', s=20)
plt.axis('equal')
plt.title("Compact Clusters", size=14)

plt.subplot(1,2,2)
plt.scatter(Xmoons[:,0], Xmoons[:,1], c='r', marker='o', s=20)
plt.axis('equal')
plt.title("Connectivity-Based Clusters", size=14)
fig = plt.gcf()
fig.set_size_inches((11,6))

We can apply the Spectral clustering on the two moons problem:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import euclidean_distances
colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
colors = np.hstack([colors] * 20)

# compute the distances
distances = euclidean_distances(Xmoons)

spectral = cluster.SpectralClustering(n_clusters=2, affinity="nearest_neighbors")
spectral.fit(Xmoons)
y_pred = spectral.labels_.astype(int)

We can now visualize the results:

In [ ]:
plt.subplot(1,2,1)
plt.scatter(Xmoons[:, 0], Xmoons[:, 1], c='r', marker='o', s=20)
plt.axis('equal')

plt.subplot(1,2,2)
plt.scatter(Xmoons[y_pred==0, 0], Xmoons[y_pred==0, 1], c='b', marker='o', s=20)
plt.scatter(Xmoons[y_pred==1, 0], Xmoons[y_pred==1, 1], c='y', marker='o', s=20)
plt.axis('equal')

fig = plt.gcf()
fig.set_size_inches((11, 6))
plt.show()

In the book, this showed a perfect clustering of the points.  In our example here, one of the horns of a moon is in the wrong cluster.  However, next time we run this, it might be correct!

We can compare the results to K-means:

In [ ]:
plt.subplot(1,2,1)
plt.scatter(Xmoons[:, 0], Xmoons[:, 1], c='r', marker='o', s=20)
plt.axis('equal')

clf = cluster.KMeans(n_clusters=2, init='k-means++')
clf.fit(Xmoons)
y_pred = clf.predict(Xmoons)

plt.subplot(1,2,2)
plt.scatter(Xmoons[y_pred==0, 0], Xmoons[y_pred==0, 1], c='b', marker='o', s=20)
plt.scatter(Xmoons[y_pred==1, 0], Xmoons[y_pred==1, 1], c='y', marker='o', s=20)
plt.axis('equal')

fig = plt.gcf()
fig.set_size_inches((11, 6))
plt.show()

As we can see, the K-means looks for spherical clusters, so it has a different result than the connection-based Spectral clusterer.